In [1]:
# Import data science libraries
import pandas as pd
from math import radians
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Matplotlib parameters
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['text.color'] = 'k'
plt.rcParams['figure.figsize'] = 12, 8

In [3]:
# Sci-kit Learn packages
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [4]:
# Read data into pandas and check features
data_dir = r'data/'
fig_dir = r'figures/'
data_file = 'WUG_SMO_Data_Clean.csv'
wind_df = pd.read_csv(data_dir + data_file)
wind_df.columns

Index(['Unnamed: 0', 'date', 'fog', 'rain', 'meanwdird', 'meanwindspdm',
       'meantempm', 'meandewptm', 'meanpressurem', 'maxhumidity',
       'minhumidity', 'maxtempm', 'mintempm', 'maxdewptm', 'mindewptm',
       'maxpressurem', 'minpressurem', 'precipm', 'year', 'month', 'day',
       'dayofweek'],
      dtype='object')

In [5]:
# Let's drop min/max and just stick with mean values for now
wind_df = wind_df.drop(['date', 'maxhumidity', 'minhumidity', 'maxtempm', 
                        'mintempm', 'maxdewptm', 'mindewptm', 'maxpressurem', 
                        'minpressurem', 'year', 'month', 'day', 'dayofweek', 'Unnamed: 0'], 
                       axis=1)

In [6]:
wind_df.head(3)

,fog,rain,meanwdird,meanwindspdm,meantempm,meandewptm,meanpressurem,precipm
0,0,1,236,3.0,18.0,14.0,1020.0,0.0
1,1,0,207,5.0,16.0,16.0,1017.0,0.0
2,1,0,251,14.0,17.0,11.0,1015.0,0.0


In [ ]:
# Rename columns for ease
wind_df.columns = ['fog', 'mdir', 'mspd', 'mtmp', 'mdew', 'mpressure', 'precipm']

# Check for null values
wind_df.isnull().sum()

In [ ]:
# Drop any rows with missing values
wind_df = wind_df.dropna()

# Check head of df
wind_df.info()

### About the data

In [ ]:
sns.set_style('white')

# Check number of foggy days
sns.countplot(x='fog', data=wind_df)

In [ ]:
# Descriptive statistics
wind_df.describe()

In [ ]:
# Check for correlation between features
sns.heatmap(wind_df.corr(), cmap='coolwarm')

In [ ]:
# We can clearly see the prevailing winds coming from a SW direction (from the ocean)
# There's also a couple interesting bumps coming from the East - the Santa Ana Winds
sns.set(style='whitegrid')
plt.figure(figsize=(8,6))

sns.kdeplot(wind_df['mdir'], shade=True)

plt.title("Distribution of Mean Wind Bearing", fontsize=18)

In [ ]:
# Let's look at how features relate to each other based on whether or not there was fog
sns.pairplot(wind_df, 'fog')
plt.savefig(fig_dir + 'pairplot_fog.png', dpi=300)

In [ ]:
# Combine these next 'target' graphs in a grid and maybe simplify with vectors

In [ ]:
plt.figure(figsize=(8,8))

ax = plt.subplot(111, polar=True)
ax.scatter(x=[radians(x) for x in wind_df['mdir'].values], 
           y=wind_df['mspd'], alpha=0.3)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)

plt.title('Mean Wind Bearing (°) vs Mean Windspeed (km/h)', fontsize=18)

In [ ]:
sns.set(style='whitegrid')
plt.figure(figsize=(8,8))

ax = plt.subplot(111, polar=True)
ax.scatter(x=[radians(x) for x in wind_df['mdir'].values], 
           y=wind_df['mdew'], alpha=0.3)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)

plt.title('Mean Wind Dir (°) vs Mean Dew Temp', fontsize=18);

In [ ]:
# Cool but realign x-grid to clearly show dual bumps, plus color fog hue

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(wind_df['mdir'], wind_df['mspd'], wind_df['mdew'], c='skyblue', s=25)
plt.xlabel('wind dir degrees')
plt.ylabel('wind spd kmh')
plt.clabel('dewpoint degrees C')
ax.view_init(25, 120)
plt.show()

### Data preprocessing

Split data into training and test sets

In [ ]:
X = wind_df[['mdir', 'mspd', 'mtmp', 'mdew', 'mpressure', 'precipm']]
y = wind_df['fog']

X[:2]

In [ ]:
# Standardize scale for model comparison
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### Modeling

In [ ]:
# Try decision tree classifer
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
dtree.score(X_test, y_test)

In [ ]:
# Run the gamute of classifiers available in scikit-learn to assess accuracy
models = [LogisticRegression(), 
          LinearSVC(), 
          SVC(kernel='rbf'), 
          KNeighborsClassifier(), 
          RandomForestClassifier(), 
          DecisionTreeClassifier(), 
          GradientBoostingClassifier(), 
          GaussianNB()]

model_names = ['Logistic Regression', 
               'Linear SVM', 
               'rbf SVM', 
               'K-Nearest Neighbors', 
               'Random Forest Classifier', 
               'Decision Tree', 
               'Gradient Boosting Classifier', 
               'Gaussian NB']

acc = []
m = {}

for model in range(len(models)):
    clf = models[model]
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc.append(accuracy_score(pred, y_test))

m = {'Algorithm':model_names, 'Accuracy':acc}

In [ ]:
acc_frame = pd.DataFrame(m)
acc_frame = acc_frame.set_index('Accuracy').sort_index(ascending=False)
acc_frame

### Logistic Regression

Instantiate model and fit data

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

Predict values and evaluate from test set

In [ ]:
pred = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))